<a href="https://colab.research.google.com/github/sirajmogcp/AgenspaceAPI/blob/main/Agentspace_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install google-api-core google-cloud-discoveryengine protobuf google-auth google-cloud-aiplatform

In [6]:
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1 as discoveryengine
from google.protobuf.json_format import MessageToDict

####
from google.oauth2.credentials import Credentials

In [7]:
def get_search_sdk_response_acl(
    creds:Credentials,
    project_id: str,
    location: str,
    engine_id: str,
    search_query: str,
    prompt_preamble: str | None = None,
) -> list[discoveryengine.SearchResponse]:
    """Retrieves search results from using the Discovery Engine Python SDK.

    Args:
        project_id: The Google Cloud project ID.
        location: The location of the Discovery Engine (e.g., "global", "us-central1").
        engine_id: The ID of the Discovery Engine.
        search_query: The search query string.
        prompt_preamble: The preamble to use for search summaries.

    Returns:
        A list of `discoveryengine.SearchResponse` objects containing the search results.
        This list may be empty if no results are found.

    """

    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    client = discoveryengine.SearchServiceClient(credentials=creds, client_options=client_options)

    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"

    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(

        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            model_prompt_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelPromptSpec(
                preamble=prompt_preamble
            ),
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="preview",
            ),
        ),
        extractive_content_spec=discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
            max_extractive_segment_count=2,
            max_extractive_answer_count=2,
            return_extractive_segment_score=True,
        ),
    )
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        content_search_spec=content_search_spec,
        # query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
        #     condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        # ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    return response



In [8]:
def get_answer_sdk_response_acl(
    creds: Credentials,
    project_id: str,
    location: str,
    engine_id: str,
    query: str,
) -> discoveryengine.AnswerQueryResponse:
    """Retrieves an answer from the Discovery Engine using the AnswerQuery API.

    This function queries the specified Discovery Engine with the provided query
    and configuration options using the `AnswerQuery` API. It supports
    specifying a location-specific endpoint for the API.

    Args:
        project_id: The Google Cloud project ID.
        location: The location of the Discovery Engine (e.g., "global", "us-central1").
        engine_id: The ID of the Discovery Engine.
        query: The query string.

    Returns:
        A `discoveryengine.AnswerQueryResponse` object containing the answer,
        citations, and other related information.

    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    """
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.ConversationalSearchServiceClient(credentials=creds,
        client_options=client_options
    )

    # The full resource name of the Search serving config
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_serving_config"

    query_understanding_spec = discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec(
        query_rephraser_spec=discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryRephraserSpec(
            disable=False,  # Optional: Disable query rephraser
            max_rephrase_steps=1,  # Optional: Number of rephrase steps
        ),
        # Optional: Classify query types
        query_classification_spec=discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryClassificationSpec(
            types=[
                discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryClassificationSpec.Type.ADVERSARIAL_QUERY,
                discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryClassificationSpec.Type.NON_ANSWER_SEEKING_QUERY,
            ]  # Options: ADVERSARIAL_QUERY, NON_ANSWER_SEEKING_QUERY or both
        ),
    )


    answer_generation_spec = discoveryengine.AnswerQueryRequest.AnswerGenerationSpec(
        ignore_adversarial_query=False,  # Optional: Ignore adversarial query
        ignore_non_answer_seeking_query=False,  # Optional: Ignore non-answer seeking query
        ignore_low_relevant_content=False,  # Optional: Return fallback answer when content is not relevant
        model_spec=discoveryengine.AnswerQueryRequest.AnswerGenerationSpec.ModelSpec(
            #model_version="gemini-2.0-flash/answer_gen/v2",  # Optional: Model to use for answer generation
        ),
        prompt_spec=discoveryengine.AnswerQueryRequest.AnswerGenerationSpec.PromptSpec(
            preamble="Give a detailed answer.",
        ),
        include_citations=True,
        answer_language_code="en",
    )

    # Initialize request argument(s)
    request = discoveryengine.AnswerQueryRequest(
        serving_config=serving_config,
        query=discoveryengine.Query(text=query),
        session=None,
        query_understanding_spec=query_understanding_spec,
        answer_generation_spec=answer_generation_spec,
    )

    # Make the request
    response = client.answer_query(request)

    return response


In [9]:

import datetime # For session expiry

# --- Vertex AI / Gemini Imports ---
import vertexai

# Using the preview namespace for safety settings as per common practice

from google.protobuf.json_format import MessageToJson

from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1 as discoveryengine
from google.protobuf.json_format import MessageToDict


In [10]:
LOCATION = "global"  # @param {type: "string"}
ENGINE_ID = ""  #  @param {type: "string", placeholder: "[your-engine-id]", isTemplate: true}
PROJECT_ID = ""  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}

In [11]:
import requests
from urllib import parse

GOOGLE_CLIENT_ID = "" #@param {type: "string", placeholder: "[OAuth ClientID]", isTemplate: true}
GOOGLE_SECRET = "" #@param {type: "string", placeholder: "[OAuth Client Secret]", isTemplate: true}
REDIRECT_URI = "https://vertexaisearch.cloud.google.com/static/oauth/oauth.html"

In [12]:
def get_authorization_url() -> str:
  """Get authorization url."""
  base_url = "https://accounts.google.com/o/oauth2/v2/auth?"
  scopes = " ".join([
      "https://www.googleapis.com/auth/calendar.events",
      "https://www.googleapis.com/auth/calendar.readonly",
      "https://www.googleapis.com/auth/userinfo.email",
      "https://www.googleapis.com/auth/drive",
      "https://www.googleapis.com/auth/cloud-platform",
  ])
  params = {
      "client_id": GOOGLE_CLIENT_ID,
      "redirect_uri": REDIRECT_URI,
      "scope": scopes,
      "include_granted_scopes": "true",
      "response_type": "code",
  }
  return base_url + parse.urlencode(params)


In [ ]:
print(get_authorization_url())
redirected_uri = input("Please paste the url you're redirected to.")

https://accounts.google.com/o/oauth2/v2/auth?client_id=&redirect_uri=https%3A%2F%2Fvertexaisearch.cloud.google.com%2Fstatic%2Foauth%2Foauth.html&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.events+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&include_granted_scopes=true&response_type=code


In [ ]:
  parsed = parse.urlparse(redirected_uri.strip().replace("#", "?"))
  authorization_code = parse.parse_qs(parsed.query)["code"][0]
  data = parse.urlencode({
      "client_id": GOOGLE_CLIENT_ID,
      "client_secret": GOOGLE_SECRET,
      "code": authorization_code,
      "grant_type": "authorization_code",
      "redirect_uri": REDIRECT_URI,
  })
  response = requests.post(
      "https://oauth2.googleapis.com/token",
      headers={"content-type": "application/x-www-form-urlencoded"},
      data=data,
  ).json()
  print(response)
  access_token = response['access_token']

In [ ]:

requests.post(
      "https://discoveryengine.googleapis.com/v1alpha/projects/811614912259/locations/global/collections/default_collection/engines/energyspace_1745599996211/servingConfigs/default_search:search",
      headers={
          "Authorization": f"Bearer {access_token}",
      }, json=dict(query="Lease")).json()

In [ ]:
  from google.oauth2.credentials import Credentials
  creds = Credentials(token=access_token)

In [ ]:
import json


In [ ]:
user_message="Lease"
response=get_search_sdk_response_acl(creds, PROJECT_ID,LOCATION,ENGINE_ID, user_message)
response=get_answer_sdk_response_acl(creds, PROJECT_ID,LOCATION,ENGINE_ID, user_message)
print(response)
